In [42]:
from word2vec import prepare_model
from elasticsearch import Elasticsearch
es = Elasticsearch()
# window=10,negative=10,min_count=3
model = prepare_model(es=es, window=15, negative=10, 
                      vector_size=300, min_count=2,
                      epochs=10)

Attempting to Load Preanalyzed Data
Training
training on 39529 docs
Saving


In [43]:
rambo = model.docvecs['7555']
star_wars = model.docvecs['11']
clone_wars = model.docvecs['12180']
empire_strikes_back = model.docvecs['1891']
return_of_the_jedi = model.docvecs['1892']
conan_the_destroyer = model.docvecs['9610']

In [44]:
model.wv.most_similar(["jedi"], topn=20)

[('darth', 0.7956092357635498),
 ('emperor', 0.745684027671814),
 ('galact', 0.7444233894348145),
 ('yoda', 0.7368442416191101),
 ('anakin', 0.7314332723617554),
 ('yavin', 0.7184299230575562),
 ('starship', 0.7172908186912537),
 ('vader', 0.7101962566375732),
 ('persian', 0.7071057558059692),
 ('kenobi', 0.7037615776062012),
 ('naboo', 0.6996074914932251),
 ('galaxi', 0.6954281330108643),
 ('pharaoh', 0.6886978149414062),
 ('shredder', 0.6829273104667664),
 ('deliver', 0.6827918887138367),
 ('wan', 0.6762895584106445),
 ('voldemort', 0.67408287525177),
 ('thor', 0.673377513885498),
 ('han_solo', 0.6728106737136841),
 ('camelot', 0.6713939905166626)]

In [51]:
model.wv.most_similar(["han_solo"], topn=40)

[('r2', 0.9097056984901428),
 ('3po', 0.9084581136703491),
 ('d2', 0.905133068561554),
 ('chewbacca', 0.8801306486129761),
 ('rames', 0.8611500263214111),
 ('spymast', 0.8527490496635437),
 ('jafar', 0.8500666618347168),
 ('banknot', 0.8361202478408813),
 ('princess_leia', 0.8318808078765869),
 ('magua', 0.8278583288192749),
 ('sarmatian', 0.826224148273468),
 ('dest', 0.8250260353088379),
 ('flu', 0.8247779607772827),
 ('apokoliptian', 0.8228570818901062),
 ('фигаро', 0.8220861554145813),
 ('milyonlarca', 0.8214942216873169),
 ('jong', 0.8200467824935913),
 ('mordor', 0.8194437026977539),
 ('aragorn', 0.8191294074058533),
 ('everdeen', 0.8189955949783325),
 ('microsoft', 0.8188924789428711),
 ('tadeusz', 0.8188700079917908),
 ('meilleur', 0.8182120323181152),
 ('magneto', 0.8179739713668823),
 ('sinn', 0.8176655173301697),
 ('garcía', 0.8176017999649048),
 ('mambo', 0.8159319758415222),
 ('bagdad', 0.8157563209533691),
 ('gru', 0.8156765699386597),
 ('parnel', 0.815586268901825),
 ('к

In [46]:
import numpy as np

model.wv.most_similar([star_wars], topn=20)

[('3po', 0.942833423614502),
 ('d2', 0.933465838432312),
 ('r2', 0.9271136522293091),
 ('jafar', 0.8995647430419922),
 ('virgina', 0.8986133933067322),
 ('schnitzler', 0.8920295238494873),
 ('rond', 0.8888728022575378),
 ('41st', 0.8875364065170288),
 ('han_solo', 0.8871241211891174),
 ('sp', 0.8850204944610596),
 ('sappho', 0.8848048448562622),
 ('meilleur', 0.884732723236084),
 ('ganesan', 0.8833495378494263),
 ('guinn', 0.8799480199813843),
 ('tadeusz', 0.878345787525177),
 ('gaull', 0.8776256442070007),
 ('magua', 0.8774573802947998),
 ('mcgiver', 0.8773785829544067),
 ('microsoft', 0.8773696422576904),
 ('renato', 0.8769243359565735)]

In [89]:
from rp import random_projection

rp_vects = []
for i in range(0,50):
    vect, lhs, rhs = random_projection(model.docvecs.vectors_docs)
    rp_vects.append(vect)
    
def rp_sim(rp_vects, vect1, vect2):
    same_sign = 0
    for rp_vect in rp_vects:
        dp1 = np.dot(rp_vect, vect1)
        dp2 = np.dot(rp_vect, vect2)
        if dp1 > 0 and dp2 > 0:
            same_sign += 1
        elif dp1 < 0 and dp2 < 0:
            same_sign += 1
    return same_sign / len(rp_vects)

In [49]:
import requests

def list_most_similar_docs(model, vect):
    for doc_id, cos_sim in model.docvecs.most_similar([vect], topn=30):
        try:
            resp = requests.get('http://localhost:9200/tmdb/movie/{}?_source=overview,title'.format(doc_id)).json()
            print(resp['_source']['title'], cos_sim)
        except e:
            print(e)
            pass

In [50]:
list_most_similar_docs(model, star_wars)

Star Wars 1.0
Arch of Triumph 0.92704176902771
The Lord of the Rings: The Two Towers 0.9133389592170715
Cleopatra 0.9067391157150269
Occupy Unmasked 0.9067099094390869
The Last Escape 0.9064509272575378
Dark of the Sun 0.9063169956207275
I Will Fight No More Forever 0.9059872627258301
The Veils of Bagdad 0.9057121276855469
Le créateur 0.9044629335403442
Macbeth 0.9025134444236755
The Tragedy of Macbeth 0.9024105072021484
The Lone Wolf in Paris 0.9023197889328003
Glitterbug 0.9009479284286499
Clash of the Titans 0.9006326794624329
Veruschka: A Life for the Camera 0.8994177579879761
The Painted Veil 0.8989959359169006
Khartoum 0.8989538550376892
Bad Grandpa 0.8980671763420105
Nouvelle Vague 0.897958517074585
The Crusades 0.8975481986999512
Moral Tales, Filmic Issues 0.897369384765625
Buried Secrets 0.8972585201263428
Ricardo O'Farrill: Abrazo Genial 0.8965938091278076
Wojaczek 0.8965892791748047
Blood Alley 0.8964524269104004
Yalom's Cure 0.8953240513801575
Naqoyqatsi 0.895049512386322
D

In [18]:
#  dissimilar
list_most_similar_docs(model, vect=np.negative(star_wars))

In [16]:
np.dot(the_dinner_game, star_wars)

-0.07022353

In [17]:
np.dot(vect, castle_in_the_sky)

0.06100053463596857

In [31]:
np.dot(rambo, star_wars)

0.13079807

In [18]:
rp_sim(rp_vects, star_wars, conan_the_destroyer)

0.84

In [19]:
rp_sim(rp_vects, star_wars, empire_strikes_back)

0.76

In [20]:
rp_sim(rp_vects, star_wars, castle_in_the_sky)

0.76

In [21]:
rp_sim(rp_vects, star_wars, rambo)

0.78

In [22]:
rp_sim(rp_vects, star_wars, the_dinner_game)

0.38

In [18]:
dir(model)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__ignoreds',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__numpys',
 '__recursive_saveloads',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__scipys',
 '__setattr__',
 '__sizeof__',
 '__slotnames__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_adapt_by_suffix',
 '_check_input_data_sanity',
 '_check_training_sanity',
 '_clear_post_train',
 '_do_train_epoch',
 '_do_train_job',
 '_get_job_params',
 '_get_offsets_and_start_doctags_for_corpusfile',
 '_get_thread_working_mem',
 '_job_producer',
 '_load_specials',
 '_log_epoch_end',
 '_log_epoch_progress',
 '_log_progress',
 '_log_train_end',
 '_raw_word_count',
 '_save_specials',
 '_set_train_params',
 '_smart_save',
 '_train_epoch',
 '_train_epoch_corpusfile',
 '_update_job_params',
 '_worker_loop',
 '_worker_loop_